In [1]:
import pandas as pd
import pickle

from data_processing import process_resstock_data, merge_building_data, create_windows

# Load data

## ResStock

In [2]:
resstock = pd.read_parquet("/Users/adrian/Documents/ICAI/4o/AI Lab/data/MA_baseline_metadata_and_annual_results.parquet")

## Buildings Timeseries

In [3]:
buildings1 = pd.read_csv("/Users/adrian/Documents/ICAI/4o/AI Lab/data/processed_buildings/buildings_tanda0.csv")

## Weather data

In [ ]:
weather_data = pd.read_csv("/Users/adrian/Documents/ICAI/4o/AI Lab/data/G2500170_2018.csv")

# Process data

## ResStock

In [4]:
columns_processed = [
    "in.bedrooms_processed",              # int
    "in.duct_leakage_processed",
    "in.duct_insulation_processed",    # Separate into leakage / insulation: X% / (Uniinsulated/R-n)
    "in.duct_location_processed",         # Places of the house
    "in.geometry_floor_area_processed",   # 1000-1499, ...
    "in.geometry_stories_processed",      # int
    "in.geometry_wall_type_processed",    # Materials
    "in.ground_thermal_conductivity_processed",   # float
    "in.hvac_has_ducts_processed",    # Yes / No
    "in.insulation_ceiling_processed",    # R-n / Uninsulated
    "in.insulation_floor_processed",      # R-n / Uninsulated
    "in.insulation_foundation_wall_processed",    # Wall R-n, Exterior
    "in.insulation_roof_processed",       # Finished / Unfinished, R-n
    "in.insulation_wall_processed",       # Material, Uninsulated / R-n
    "in.occupants_processed",         # int
    "in.orientation_processed",       # North, Northwest...
    "in.roof_material_processed",     # Material
    "in.sqft_processed",              # int
    "in.windows_processed", # Single/Double/Triple, Low-E/Clear, Metal/Non-metal, Air/Exterior Clear Storm, (L/M-Gain)
    "in.window_areas_processed",
    "in.vintage_processed",
    
]

In [5]:
resstock_processed = process_resstock_data(resstock)
resstock_processed = resstock_processed[columns_processed]

In [ ]:
windows = create_windows(buildings1, resstock_processed, window_size=2)

In [ ]:
# Save to pickle file
with open("/Users/adrian/Documents/ICAI/4o/AI Lab/data/windows1.pkl", "wb") as f:
    pickle.dump(windows, f)

## Weather

In [ ]:
# Change date time column name to timestamp
weather_data.rename(columns={'date_time': "timestamp"}, inplace=True)
weather_data.set_index('timestamp', inplace=True)
weather_data.index = pd.to_datetime(weather_data.index)

In [ ]:
# Delete all columns except dry bulb temperature and direct normal radiation
weather_data = weather_data[["Dry Bulb Temperature [°C]", "Direct Normal Radiation [W/m2]"]]
weather_data.rename(columns={"Dry Bulb Temperature [°C]": "temperature"}, inplace=True)

In [ ]:
# Create an entry for 2018-01-01 00:00:00
new_row = pd.DataFrame({'temperature': weather_data.iloc[0]['temperature']}, index=[pd.to_datetime('2018-01-01 00:00:00')])
weather_data = pd.concat([new_row, weather_data])

In [ ]:
weather_data.bfill(inplace=True)
weather_data.ffill(inplace=True)